In [1]:
import math
import requests
import numpy as np

pricelist_url = "https://cloudpricingcalculator.appspot.com/static/data/pricelist.json"

In [2]:
r = requests.get(pricelist_url)
pricelist = r.json()
print("Pricing as of {}".format(pricelist['updated']))

Pricing as of 05-August-2020


In [3]:
class HostedCourse(dict):
    def __init__(self, data, pricelist, **kw):
        dict.__init__(self)
        self.update(data)
        self.pricelist = pricelist
        self.update(kw['update'])

    def monthly_cost_all_pd(self):
        pd_total_size = self['num_users'] * self['pd_size_gb']
        pd_rate_gb_month = self.pricelist['gcp_price_list'][self['pd_type']][self['node_region']]
        return pd_rate_gb_month * pd_total_size

    def monthly_cost_node(self):
        node_rate = self.pricelist['gcp_price_list'][self['node_type']][self['node_region']]
        return node_rate * self['sustained_use_factor'] * self['node_monthly_uptime_h']

    def pods_per_node(self):
        mem_per_node = int(self.pricelist['gcp_price_list'][self['node_type']]['memory'])
        cores_per_node = int(self.pricelist['gcp_price_list'][self['node_type']]['cores'])
        return math.floor((mem_per_node-1)/self['mem_per_pod_gb'])
    
    def derived_node_count(self):
        return math.ceil(self['num_active_pods'] / self.pods_per_node())
    
    def monthly_cost_all_nodes(self):
        return self.derived_node_count() * self.monthly_cost_node()
    
    def monthly_cost_total(self):
        return self.monthly_cost_all_nodes() + self.monthly_cost_all_pd()
    
    def monthly_cost_per_student(self):
        return self.monthly_cost_total() / self['num_users']
    
    def cores_per_user(self):
        return self.derived_node_count()*int(self.pricelist['gcp_price_list'][self['node_type']]['cores']) / self['num_active_pods']
    
    def mem_per_user(self):
        return self.derived_node_count()*int(self.pricelist['gcp_price_list'][self['node_type']]['memory']) / self['num_active_pods']

    

In [9]:
# defaults 
dsep = {
    ## GKE
    'node_region': 'europe',

    # persistent disk
    'pd_type': 'CP-COMPUTEENGINE-STORAGE-PD-SSD',
    'node_type': 'CP-COMPUTEENGINE-VMIMAGE-N1-STANDARD-4',
    'num_users': 1,
    'num_active_pods': 1,
    'pd_size_gb': 1,
    'node_monthly_uptime_h': 30*24,
    'mem_per_pod_gb': 0.5,
    'sustained_use_factor': 0.85, #What is correct value?
}

In [5]:
def show_node_info(nt):
    '''Display a node type's hourly rate and it's memory and core counts.'''
    print("rate: {:.3f}, mem_gb: {}, cores: {}".format(
        pricelist['gcp_price_list'][nt]['europe'],
        pricelist['gcp_price_list'][nt]['memory'],
        pricelist['gcp_price_list'][nt]['cores']
    ))

In [6]:
show_node_info('CP-COMPUTEENGINE-VMIMAGE-N1-STANDARD-4')

rate: 0.209, mem_gb: 15, cores: 4


In [18]:
courses = {
    'course': HostedCourse(dsep, pricelist, update={
        'num_users': 500, #Total number of users
        'mem_per_pod_gb': 1, #memory pr user
        'pd_size_gb': 0.5, #persistent storage pr user
        'num_active_pods': 100, #number of simultanious users
        
    }),
}

In [19]:
for k in sorted(courses.keys()):
    course = courses[k]
    print("{}\ntotal pr month: ${:8.2f}\npr user: ${:6.2f}\nnumber of nodes: {}\ncores pr user: {}\nmemory pr user: {}GB".format(
        k,
        course.monthly_cost_total(),
        course.monthly_cost_per_student(),
        course.derived_node_count(),
        course.cores_per_user(),
        course.mem_per_user()
    ))

course
total pr month: $ 1066.74
pr user: $  2.13
number of nodes: 8
cores pr user: 0.32
memory pr user: 1.2GB
